<a href="https://colab.research.google.com/github/borgesf/Fomula_Magica_Greenblatt/blob/main/Magic_Formula_Greenblatt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fórmula Mágica - Joel Greenblat

### Autor: Filipe Borges (f3l5p7@yahoo.com.br)
### [Fonte](https://www.youtube.com/watch?v=e_ZRDG4F4ZA&list=PLCAhGm8nJ9CBn51o0x3j1p1LuMRqpeqCy&index=8)
### Data da Versão: 23/04/2021


\\

Notebook para gerar um *ranking* de ações da Bovespa, seguinte critérios de liquidez diária, ROIC e P/L. Fonte dos dados: [Fundamentus](https://www.fundamentus.com.br)

## 1 - Importando Bibliotecas e Carregando Dados

In [2]:
import pandas as pd
import requests
# import string

# import warnings
# warnings.filterwarnings('ignore')

url = 'http://www.fundamentus.com.br/resultado.php'
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

# Pre-processamento dos dados: troca ',' por '.' em alguns decimais, e converte strings para float

for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype(float) / 100

## Filtros de liquidez e "qualidade" - P/L e ROE

In [ ]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - P/L (>0) e ROE
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['P/L'] = df[df['P/L'] > 0].sort_values(by=['P/L'])['Papel'][:150].values
ranking['ROE'] = df.sort_values(by=['ROE'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='P/L', values='position')
aux_b = ranking.pivot_table(columns='ROE', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:15]

## Filtros de liquidez e "qualidade" alternativos - EV/EBIT e ROIC (NÃO VALE PARA FINANCEIRAS E BANCOS!!!)

In [ ]:
# Filtro de Liquidez: > 1 Milhão/2 meses
df = df[df['Liq.2meses'] > 1000000]

# Rankeamento - EV/EBIT (>0) e ROIC
ranking = pd.DataFrame()
ranking['position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

# Variaveis auxiliares para gerar ranking final - Imprime os 15 primeiros
aux_a = ranking.pivot_table(columns='EV/EBIT', values='position')
aux_b = ranking.pivot_table(columns='ROIC', values='position')
aux_full=pd.concat([aux_a,aux_b])
Final_Rank = aux_full.dropna(axis=1).sum()
Final_Rank.sort_values()[:15]